### Load data

In [ ]:
# configure notebook for displaying plots 
%matplotlib inline

# numeric and data wrangling
import numpy as np
import pandas as pd

In [ ]:
path_npy_data = '../data_BHsim/data_BHsim_500split100_npy/'

In [ ]:
# reset variables
X_train = []
y_train = []
X_test = []
y_test = []

X_train = np.load(path_npy_data + 'X_train_BlackHoleMilkyWay1_1687133209.npy', allow_pickle=True) 
y_train = np.load(path_npy_data + 'y_train_BlackHoleMilkyWay1_1687133209.npy', allow_pickle=True) 
X_test = np.load(path_npy_data + 'X_test_BlackHoleMilkyWay1_1687133209.npy', allow_pickle=True) 
y_test = np.load(path_npy_data + 'y_test_BlackHoleMilkyWay1_1687133209.npy', allow_pickle=True) 

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
#print(X_train.shape, y_train.shape)
y_train[0:50] # look for random sequence, means data is shuffled

### Define CNN

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [ ]:
inputs = keras.Input(shape=(180, 360, 3)) # BH_sim
#inputs = keras.Input(shape=(299, 299, 3))
x = data_augmentation(inputs)
#x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for d in devices:
    t = d.device_type
    name = d.physical_device_desc
    l = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==2])
    dev = name_attr.get('name', 'Unnamed device')
    print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")

In [ ]:
import tensorflow as tf

In [ ]:
GPUS = ["GPU:0"]

In [ ]:
strategy = tf.distribute.MirroredStrategy( GPUS )
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

In [ ]:
EPOCHS = 10 #50 #8 #4 #25 #2 #1 #100 #5 #1 #5 #3
BATCH_SIZE = 8 # to match referece paper #5

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

print(BATCH_SIZE, batch_size)

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy', tf.keras.metrics.Recall(), 
                       tf.keras.metrics.Precision(), tf.keras.metrics.AUC()]
             )

In [ ]:
%%time

history_curve = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data = (X_test,y_test))

In [ ]:
df_hist = pd.DataFrame(history_curve.history)
df_hist = df_hist.reset_index()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(7,5))
plt.style.use('default')
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-poster')
plt.plot(df_hist['index'], df_hist['accuracy'], '--', label='train_accuracy')
plt.plot(df_hist['index'], df_hist['val_accuracy'],'--', label='val_accuracy')
plt.xlabel('Training Epoch')
plt.ylabel('Accuracy')
plt.legend();

In [ ]:
df_hist.to_csv('df_hist_2023-06-18a.csv', index=False)

In [ ]:
model.save('trained_tf_model_BHSim1_2023-06-18a.h5')